インポート

In [1]:
# 必要なライブラリのインポート
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pprint import pprint

スクレイピングするURL

In [2]:
# schooのURLを設定する
#カテゴリごとになっているURL
url_content = "https://schoo.jp/class/category/dx/427?page={}"
category_url = url_content.format(1)

In [3]:
# レスポンスを取得する
res = requests.get(category_url)

# 取得結果をsoupに格納する
soup = BeautifulSoup(res.text, 'html.parser', from_encoding='utf-8')

/Users/yakushijimana/anaconda3/envs/reskill/lib/python3.11/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [4]:
#カテゴリとして講義内容を取ってくる
contents = soup.find_all('div', class_='scrollContent listItem')


In [5]:
# 取得できていることを確認する
len(contents)

42

In [6]:
content = contents[1]
print(content)

<div class="scrollContent listItem">
<a class="ga_link" data-ga='{"category":"categoryInfinityScrollContent", "position":"class10715"}' href="/course/7855" title="ITパスポート資格試験対策　〜問題演習編〜（令和５年度対応）">
<div class="parent_wrap">
<div class="images_wrap">
<div class="images">
<img alt="ITパスポート資格試験対策　〜問題演習編〜（令和５年度対応）" class="lazyload" data-src="//s3-ap-northeast-1.amazonaws.com/i.schoo/images/class/600x260/10715.jpg" src="//s3-ap-northeast-1.amazonaws.com/i.schoo/images/common/spacer.svg"/>
</div>
</div>
</div>
<div class="info">
<div class="info__flexbox">
<div>
<h3 class="title"><span class="badge blue">全3回</span>ITパスポート資格試験対策　〜問題演習編〜（令和５年度対応）</h3>
<p class="date">
                    2023年9月28日公開                  </p>
</div>
<div class="bookmark">
<div class="reservation mark" data-course-id="7855">
<div class="reservationBox" class_id="10715"></div>
</div>
</div>
</div>
<div class="detail">
<p class="description">
                  ITに関する基礎知識を身につける

ITパスポート試験はITを利活用するすべての社会人・これから社会人となる     

In [7]:
#講義内容の詳細へとべるリンク
a_tag = content.find('a')
url = a_tag.get('href')
print(url)
#使うときは以下の形でリンクに飛べる
print("https://schoo.jp"+url)
link = "https://schoo.jp"+url

/course/7855
https://schoo.jp/course/7855


In [8]:
#講義名
big_title = content.find("h3", class_="title").text
big_title


'全3回ITパスポート資格試験対策\u3000〜問題演習編〜（令和５年度対応）'

In [9]:
#関連ワード
# 関連ワードを格納するリスト
related_words = []

# .find_all()で取得した結果をループ
for words in content.find_all("a", class_="tag ga_link"):
    # 各要素からテキストを取得し、リストに追加
    related_words.append(words.text)
# リスト内の各文字列から "n/" と空白を削除する
related_words = [word.strip('\n ') for word in related_words]

# リストを表示
print(related_words)


['ITリテラシー', 'デジタルリテラシー']


リンク先（講義情報）へアクセスし詳細情報を取ってくる

In [10]:
# linkにアクセスしてHTMLを取得
detail = requests.get(link)

# ステータスコードが200（成功）でない場合はエラーを出力して終了
if detail.status_code != 200:
    print(f"Error: Failed to fetch the page. Status code: {detail.status_code}")
    exit()

# 取得したHTMLをBeautifulSoupで解析
soup_detail = BeautifulSoup(detail.text, 'html.parser')

In [11]:
#コース概要
agenda = soup_detail.find("strong").text
agenda

'ITに関する基礎知識を身につける'

In [12]:
#講師名
teacher_name = soup_detail.find("h4", class_="p-courseTeachers__columnName").text
#現職
teacher_post = soup_detail.find("p", class_="p-courseTeachers__columnPost").text
#経歴
teacher_profile = soup_detail.find("p", class_="p-courseTeachers__columnProfile").text

In [13]:
#講義詳細
#講義リンク
lecture_link_pre = soup_detail.find_all("li", class_="p-classes__item js-foldingByItemCountItem")
# lecture_link_pre内の各要素からaタグを見つけてそのhref属性を取得
lecture_links = [item.find('a').get('href') for item in lecture_link_pre]

# 各要素に"https://schoo.jp"を前に追加して新しいリストに格納
lecture_url = ["https://schoo.jp" + link for link in lecture_links]


#講義タイトル
middle_title_pre = soup_detail.find_all("h3", class_="p-classes__title")
# middle_title_preから各要素のテキストを取得
middle_titles = [middle_title_pre_1.text.strip() for middle_title_pre_1 in middle_title_pre]


# 講義時間
lecture_time_pre = soup_detail.find_all("span", class_="p-classes__onAirDuration")
lecture_times = [lecture_time.text.strip() for lecture_time in lecture_time_pre]




In [14]:
dic = {
    "link": link,
    "big_title": big_title,
    "agenda": agenda,
    "lecture_link": lecture_url,
    "middle_titles": middle_titles,
    "lecture_time": lecture_times,
    "teacher_name": teacher_name,
    "teacher_post": teacher_post,
    "teacher_profile": teacher_profile,
    "related_words": related_words,
}

In [15]:
dic

{'link': 'https://schoo.jp/course/7855',
 'big_title': '全3回ITパスポート資格試験対策\u3000〜問題演習編〜（令和５年度対応）',
 'agenda': 'ITに関する基礎知識を身につける',
 'lecture_link': ['https://schoo.jp/class/10715',
  'https://schoo.jp/class/10716',
  'https://schoo.jp/class/10717'],
 'middle_titles': ['1.設問1~31', '2.設問32~65', '3.設問66~100'],
 'lecture_time': ['65min', '70min', '75min'],
 'teacher_name': '石田 宏実',
 'teacher_post': 'デジタルプランニング株式会社 代表取締役',
 'teacher_profile': 'デジタルプランニング株式会社 代表。udemy講師（開講から１年で受講者１８０００人、延数２５０００人）。応用情報技術者試験、データベーススペシャリスト試験合格。著書に「ゲーミフィケーション」（大和出版）。８年間のサラリーマンプログラマを経て、２００７年に独立。２００９年にデジタルプランニング株式会社設立。ネット集客の知識はもちろん、ネットを使わないいわゆる「リアル」集客も得意とする。',
 'related_words': ['ITリテラシー', 'デジタルリテラシー']}

for文を回す

In [16]:
#d_listに空リストを作成する
d_list = []

# schooのURLを設定する
#カテゴリごとになっているURL
url_content = "https://schoo.jp/class/category/dx/427?page={}"
category_url = url_content.format(1)

# レスポンスを取得する
res = requests.get(category_url)

# 取得結果をsoupに格納する
soup = BeautifulSoup(res.text, 'html.parser', from_encoding='utf-8')

#カテゴリとして講義内容を取ってくる
contents = soup.find_all('div', class_='scrollContent listItem')

# for文で各情報を取得する
for content in contents:
    #講義内容の詳細へとべるリンク
    a_tag = content.find('a')
    url = a_tag.get('href')
    #使うときは以下の形でリンクに飛べる
    link = "https://schoo.jp"+url
    #講義大分類名
    big_title = content.find("h3", class_="title").text
    #関連ワード
    # 関連ワードを格納するリスト
    related_words = []

    # .find_all()で取得した結果をループ
    for words in content.find_all("a", class_="tag ga_link"):
        # 各要素からテキストを取得し、リストに追加
        related_words.append(words.text)
    # リスト内の各文字列から "n/" と空白を削除する
    related_words = [word.strip('\n ') for word in related_words]
    
    # linkにアクセスしてHTMLを取得
    detail = requests.get(link)
    # ステータスコードが200（成功）でない場合はエラーを出力して終了
    if detail.status_code != 200:
        print(f"Error: Failed to fetch the page. Status code: {detail.status_code}")
        exit()
    # 取得したHTMLをBeautifulSoupで解析
    soup_detail = BeautifulSoup(detail.text, 'html.parser')

    #コース概要
    #agenda_element = soup_detail.find("strong")
    # 要素が存在する場合は text 属性を取得、存在しない場合はデフォルトの値を使用
    #agenda = agenda_element.text if agenda_element else "Not available"


    #講師名
    teacher_name = soup_detail.find("h4", class_="p-courseTeachers__columnName").text
    #現職
    teacher_post = soup_detail.find("p", class_="p-courseTeachers__columnPost").text
    #経歴
    teacher_profile = soup_detail.find("p", class_="p-courseTeachers__columnProfile").text

    #講義詳細
    #講義リンク
    lecture_link_pre = soup_detail.find_all("li", class_="p-classes__item js-foldingByItemCountItem")
    # lecture_link_pre内の各要素からaタグを見つけてそのhref属性を取得
    lecture_links = [item.find('a').get('href') for item in lecture_link_pre]

    # 各要素に"https://schoo.jp"を前に追加して新しいリストに格納
    lecture_url = ["https://schoo.jp" + link for link in lecture_links]


    #講義タイトル
    middle_title_pre = soup_detail.find_all("h3", class_="p-classes__title")
    # middle_title_preから各要素のテキストを取得
    middle_titles = [middle_title_pre_1.text.strip() for middle_title_pre_1 in middle_title_pre]


    # 講義時間
    lecture_time_pre = soup_detail.find_all("span", class_="p-classes__onAirDuration")
    lecture_times = [lecture_time.text.strip() for lecture_time in lecture_time_pre]

    dic = {
        "link": link,
        "big_title": big_title,
        #"agenda": agenda,
        "lecture_link": lecture_url,
        "middle_titles": middle_titles,
        "lecture_time": lecture_times,
        "teacher_name": teacher_name,
        "teacher_post": teacher_post,
        "teacher_profile": teacher_profile,
        "related_words": related_words,
    }

    d_list.append(dic)

In [17]:
len(d_list)

42

In [18]:
max = range(0, len(contents))
df =pd.DataFrame(d_list)

In [19]:
df

,link,big_title,lecture_link,middle_titles,lecture_time,teacher_name,teacher_post,teacher_profile,related_words
0,https://schoo.jp/course/7883,全1回Microsoft 365 Copilotが業務にもたらす未来,[https://schoo.jp/class/10777],[Microsoft 365 Copilotって何ができるの？],[40min],臼井 拓水,Michikusa株式会社 代表取締役,AIの研究開発を行うPKSHA Technologyを母体としたファンドPKSHA Capi...,"[Excel, PowerPoint, Word, 資料作成, AI活用, ITリテラシー,..."
1,https://schoo.jp/course/7855,全3回ITパスポート資格試験対策 〜問題演習編〜（令和５年度対応）,"[https://schoo.jp/class/10715, https://schoo.j...","[1.設問1~31, 2.設問32~65, 3.設問66~100]","[65min, 70min, 75min]",石田 宏実,デジタルプランニング株式会社 代表取締役,デジタルプランニング株式会社 代表。udemy講師（開講から１年で受講者１８０００人、延数２...,"[ITリテラシー, デジタルリテラシー]"
2,https://schoo.jp/course/7783,全3回Shunsuke式 探求ChatGPT,"[https://schoo.jp/class/10557, https://schoo.j...","[生産性向上ツールをChatGPTで発明しよう, コミュニケーション向上ツールをChatGP...","[60min, 60min, 60min]",林 駿甫 (Shunsuke),プロンプトアーティスト,AIと共生する未来を目指すプロンプトアーティスト。共著『基礎から学ぶノーコード開発』(C &...,"[時間術, 資料作成, 業務効率化, 課題解決力, DX基礎, ITリテラシー, デジタルリ..."
3,https://schoo.jp/course/7713,全1回誰一人取り残さないGA4相談会,[https://schoo.jp/class/10400],[誰一人取り残さないGA4相談会],[60min],ヨシミ,ブログコンサルタント,30歳で夢見てきたミュージシャンを諦め、ブラック就職。\n働いても働いても改善されない給料や...,"[Google Analytics, アクセス解析, DX基礎, デジタルリテラシー]"
4,https://schoo.jp/course/7729,全10回ITパスポート資格試験対策 〜講義編〜（令和５年度対応）,"[https://schoo.jp/class/10464, https://schoo.j...","[1.情報システム開発, 2.プロジェクトマネジメント, 3.サービスマネジメント／システム...","[60min, 60min, 60min, 60min, 60min, 70min, 70m...",石田 宏実,デジタルプランニング株式会社 代表取締役,デジタルプランニング株式会社 代表。udemy講師（開講から１年で受講者１８０００人、延数２...,"[ITリテラシー, デジタルリテラシー]"
5,https://schoo.jp/course/7654,全1回今日から始めるChatGPT,[https://schoo.jp/class/10329],[AIをうまく使う「頼み方」のコツ],[60min],小澤 健祐,ディップ株式会社 AINOW 編集長,「人間とAIが共存する社会をつくる」がビジョン。持続的な次世代社会を創るべく、コミュニケーシ...,"[AI活用, ITリテラシー, デジタルリテラシー]"
6,https://schoo.jp/course/7668,全1回AI時代のクリエイターのキャリア,[https://schoo.jp/class/10355],[AI時代のクリエイターのキャリア],[60min],ブランドン・K・ヒル,"Founder & CEO, btrax, Inc.",サンフランシスコ州立大学デザイン科卒。サンフランシスコと東京に拠点を持つデザイン会社btra...,"[デザイン, AI開発, 機械学習, デジタル社会, デジタルリテラシー]"
7,https://schoo.jp/course/7637,全1回ビッグデータ 基礎理解と企業の活用事例,[https://schoo.jp/class/10289],[ビッグデータ 基礎理解と企業の活用事例],[60min],小澤 健祐,ディップ株式会社 AINOW 編集長,「人間とAIが共存する社会をつくる」がビジョン。持続的な次世代社会を創るべく、コミュニケーシ...,"[DX基礎, デジタルリテラシー]"
8,https://schoo.jp/course/7596,全1回ChatGPTでシンプルなゲームを作ろう,[https://schoo.jp/class/10215],[ChatGPTでシンプルなゲームを作ろう],[60min],平岡 拓,定時で帰るための「個人DX」コンサルタント,コードを書かないプログラミング「ノーコード」を専門とするDXコンサルタントです。エンジニアで...,"[AI活用, ITリテラシー, デジタルリテラシー]"
9,https://schoo.jp/course/7585,全35回その作業はAIにやらせてみよう,"[https://schoo.jp/class/10211, https://schoo.j...","[ChatGPTでメールの文章をシュッと短くしよう, ChatGPTを情報収集の助手にしよう...","[40min, 40min, 40min, 40min, 40min, 40min, 40m...",古川 渉一,株式会社デジタルレシピCTO,1992年鹿児島生まれ。東京大学工学部卒業。株式会社デジタルレシピ取締役CTO。パワーポイン...,"[API, イラスト制作, クリエイティブデザイン, 企画提案, 情報活用, AI開発, 業..."


csvファイルに保存

In [20]:
# csvファイルに出力する
df.to_csv("schoo_v1.csv", index = False)

複数のページから取得する（一つのカテゴリ内において）